In [ ]:
from dateutil import parser, tz
from datetime import datetime
import requests
import csv
import pytz

# Configuration
wfo = "CLE"
start_date = "2024-01-01T00:00:00Z"
end_date = "2024-08-21T00:00:00Z"
priority_county = "Cuyahoga OH"

def utc_to_cdt(utc_dt):
    if isinstance(utc_dt, str):
        utc_dt = parser.parse(utc_dt)
    utc_dt = utc_dt.replace(tzinfo=tz.tzutc())
    cdt_dt = utc_dt.astimezone(tz.gettz('America/Chicago'))  # Convert to CDT
    return cdt_dt.strftime('%Y-%m-%d %I:%M:%S %p')  # Return as string in readable format, including AM/PM

def get_detailed_warning_data(wfo, year, phenomena, etn, significance):
    api_url = f"https://mesonet.agron.iastate.edu/json/vtec_event.py?wfo={wfo}&year={year}&phenomena={phenomena}&etn={etn}&significance={significance}"
    print(f"Fetching detailed warning data: {api_url}")
    response = requests.get(api_url)
    if response.status_code == 200:
        data = response.json()
        if "ugcs" in data and len(data["ugcs"]) > 0 and "utc_init_expire" in data["ugcs"][0]:
            print(f"Found initial expire time for ETN {etn}")
            return data["ugcs"][0]["utc_init_expire"]
        else:
            print(f"Initial expire time not found for ETN {etn}")
            return None
    else:
        print(f"Failed to fetch detailed data for ETN {etn}, status code: {response.status_code}")
        if response.status_code == 422:
            print(f"Error details: {response.json()}")
        return None

def get_tornado_warnings(wfo, start_date, end_date):
    api_url = f"https://mesonet.agron.iastate.edu/api/1/cow.json?wfo={wfo}&begints={start_date}&endts={end_date}&phenomena=TO&lsrtype=T&lsrbuffer=15&warningbuffer=1"
    print(f"Fetching tornado warnings: {api_url}")
    response = requests.get(api_url)
    if response.status_code == 200:
        print("Tornado warnings fetched successfully")
        return response.json()
    else:
        print(f"Failed to fetch data from the API, status code: {response.status_code}")
        return None

def calculate_duration(start_time, init_expire_time):
    start_time_cdt = utc_to_cdt(start_time)
    expire_time_cdt = utc_to_cdt(init_expire_time)
    duration_seconds = int((parser.parse(expire_time_cdt) - parser.parse(start_time_cdt)).total_seconds())
    duration_minutes = duration_seconds // 60
    return f"{duration_minutes}"

def save_to_csv(warnings, filename, priority_county=None):
    priority_county = priority_county.lower()  # Ensure case-insensitivity
    unique_counties = set()

    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['WFO', 'Issuance Date', 'Issuance Time', 'Initial Expire Time', 'Verified', 'Counties', 'Time Active (minutes)'])

        for feature in warnings['events']['features']:
            counties = ", ".join(feature['properties']['ar_ugcname'])
            unique_counties.update(feature['properties']['ar_ugcname'])  # Collect unique county names
           # print(f"Processing counties: {counties}")
            if priority_county in counties.lower():
                wfo = feature['properties']['wfo']
                issue_time_utc = feature['properties']['issue']
                year = parser.parse(issue_time_utc).year
                etn = feature['properties']['eventid']
                print(f"Processing ETN: {etn}")
                init_expire_time = get_detailed_warning_data(wfo, year, "TO", etn, "W")
                if init_expire_time:
                    issue_time_cdt = utc_to_cdt(issue_time_utc)
                    expire_time_cdt = utc_to_cdt(init_expire_time)
                    duration = calculate_duration(issue_time_utc, init_expire_time)
                    verify = feature['properties'].get('verify', 'N/A')
                    writer.writerow([wfo, issue_time_cdt.split(' ')[0], ' '.join(issue_time_cdt.split(' ')[1:]), ' '.join(expire_time_cdt.split(' ')[1:]), verify, counties, duration])
                    print(f"Written ETN {etn} to CSV")
                else:
                    print(f"Skipping ETN {etn} due to missing initial expire time")
            else:
                print(f"Skipping due to no match in priority county: {priority_county}")

    print(f"Unique counties processed: {unique_counties}")

warnings = get_tornado_warnings(wfo, start_date, end_date)
if warnings:
    print("Tornado warnings data:")
    print(warnings)
    print("Saving warnings to CSV")
    csv_filename = "tornado_warnings.csv"
    save_to_csv(warnings, csv_filename, priority_county)
    print("Data saved to CSV.")
else:
    print("No warnings to save")


Fetching tornado warnings: https://mesonet.agron.iastate.edu/api/1/cow.json?wfo=CLE&begints=2024-01-01T00:00:00Z&endts=2024-08-21T00:00:00Z&phenomena=TO&lsrtype=T&lsrbuffer=15&warningbuffer=1
Tornado warnings fetched successfully
Tornado warnings data:
{'generated_at': '2024-08-22T17:00:45Z', 'params': {'wfo': ['CLE'], 'phenomena': ['TO'], 'lsrtype': ['T'], 'hailsize': 1, 'lsrbuffer': 15.0, 'wind': 58, 'windhailtag': False, 'limitwarns': False, 'begints': '2024-01-01T00:00:00Z', 'endts': '2024-08-21T00:00:00Z', 'warningbuffer': 1.0}, 'stats': {'area_verify[%]': 15.187077471319348, 'shared_border[%]': 39.95781701169426, 'max_leadtime[min]': 24, 'min_leadtime[min]': 2, 'avg_leadtime[min]': 15.0, 'avg_leadtime_firstreport[min]': 10.5, 'tdq_stormreports': 0, 'unwarned_reports': 4, 'warned_reports': 12, 'events_verified': 8, 'events_total': 33, 'reports_total': 16, 'POD[1]': 0.75, 'FAR[1]': 0.7575757575757576, 'CSI[1]': 0.22429906542056077, 'avg_size[sq km]': 804.4248524094779, 'size_poly_v

In [ ]:
# @title Broken, do not use
#the code now only displays the warnings in the county we write below, if you want all counties in the WFO use the old code below
#CDT/CST aware/printed

from dateutil import parser, tz
from datetime import datetime
import requests
import csv
import pytz

# Configuration
wfo = "DTX"
start_date = "2022-01-01T00:00:00Z"
end_date = "2023-07-31T00:00:00Z"
priority_county = "Wayne MI"

def utc_to_cdt(utc_dt):
    if isinstance(utc_dt, str):
        utc_dt = parser.parse(utc_dt)
    utc_dt = utc_dt.replace(tzinfo=tz.tzutc())
    cdt_dt = utc_dt.astimezone(tz.gettz('America/Chicago'))  # Convert to CDT
    return cdt_dt.strftime('%Y-%m-%d %I:%M:%S %p')  # Return as string in readable format, including AM/PM
def utc_to_cdt(utc_dt):
    # Ensure utc_dt is a datetime object
    if isinstance(utc_dt, str):
        utc_dt = parser.parse(utc_dt)

    # Make timezone-aware as UTC
    utc_dt = utc_dt.replace(tzinfo=tz.tzutc())

    # Convert to CDT, using 'America/Chicago'
    target_tz = tz.gettz('America/Chicago')
    cdt_dt = utc_dt.astimezone(target_tz)

    # Format datetime string to include timezone abbreviation
    # The '%Z' directive outputs the timezone name (e.g., CST, CDT)
    return cdt_dt.strftime('%Y-%m-%d %I:%M:%S %p %Z')

def get_detailed_warning_data(wfo, year, phenomena, etn, significance):
    api_url = f"https://mesonet.agron.iastate.edu/json/vtec_event.py?wfo={wfo}&year={year}&phenomena={phenomena}&etn={etn}&significance={significance}"
    response = requests.get(api_url)
    if response.status_code == 200:
        data = response.json()
        if "ugcs" in data and len(data["ugcs"]) > 0 and "utc_init_expire" in data["ugcs"][0]:
            return data["ugcs"][0]["utc_init_expire"]
        else:
            print(f"Initial expire time not found for ETN {etn}")
            return None
    else:
        print(f"Failed to fetch detailed data for ETN {etn}")
        return None

def get_tornado_warnings(wfo, start_date, end_date):
    api_url = f"https://mesonet.agron.iastate.edu/api/1/cow.json?wfo={wfo}&begints={start_date}&endts={end_date}&phenomena=TOR&lsrtype=T&lsrbuffer=15&warningbuffer=1"
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to fetch data from the API")
        return None

def calculate_duration(start_time, init_expire_time):
    start_time_cdt = utc_to_cdt(start_time)
    expire_time_cdt = utc_to_cdt(init_expire_time)
    duration_seconds = int((parser.parse(expire_time_cdt) - parser.parse(start_time_cdt)).total_seconds())
    duration_minutes = duration_seconds // 60
    return f"{duration_minutes}"

def save_to_csv(warnings, filename, priority_county=None):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['WFO', 'Issuance Date', 'Issuance Time', 'Initial Expire Time', 'Verified', 'Counties', 'Time Active (minutes)'])

        for feature in warnings['events']['features']:
            counties = ", ".join(feature['properties']['ar_ugcname'])
            if priority_county in counties:
                wfo = feature['properties']['wfo']
                issue_time_utc = feature['properties']['issue']
                year = parser.parse(issue_time_utc).year
                etn = feature['properties']['eventid']
                init_expire_time = get_detailed_warning_data(wfo, year, "TO", etn, "W")
                if init_expire_time:
                    issue_time_cdt = utc_to_cdt(issue_time_utc)
                    expire_time_cdt = utc_to_cdt(init_expire_time)
                    duration = calculate_duration(issue_time_utc, init_expire_time)
                    verify = feature['properties'].get('verify', 'N/A')
                    writer.writerow([wfo, issue_time_cdt.split(' ')[0], ' '.join(issue_time_cdt.split(' ')[1:]), ' '.join(expire_time_cdt.split(' ')[1:]), verify, counties, duration])

warnings = get_tornado_warnings(wfo, start_date, end_date)
if warnings:
    csv_filename = "tornado_warnings.csv"
    save_to_csv(warnings, csv_filename, priority_county)
    print("Data saved to CSV.")

In [ ]:
# @title Test code, do not use


import requests
import csv
from datetime import datetime
from dateutil import parser

#fill these out with the same format for whichever area you want
wfo = "DTX"
start_date = "2022-01-01T00:00:00Z" #the T and Z HAVE to be here
end_date = "2024-07-31T23:00:00Z"
priority_county = "Wayne MI"  # The county to be sorted first, MUST PUT STATE ABBREVIATION AFTERWARDS WITH NO COMMA, will default to alphabetical order if no county is specified, write "" for no county specified

def get_tornado_warnings(wfo, start_date, end_date):
    api_url = f"https://mesonet.agron.iastate.edu/api/1/cow.json?wfo={wfo}&begints={start_date}&endts={end_date}&phenomena=TOR&lsrtype=T&lsrbuffer=15&warningbuffer=1" #this pulls the data from the iem cow api
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to fetch data from the API")
        return None

def calculate_duration(start_time, end_time): #this does the calculation using the issue time and expire time of each nws warning to show the time they were active
    duration_seconds = int((end_time - start_time).total_seconds())
    duration_minutes = duration_seconds // 60
    return f"{duration_minutes}" if duration_minutes != 1 else "1" #if they issue something with a messed up time it will default to 1 minute
def save_to_csv(warnings, filename, priority_county=None):
    # Sort function to prioritize the presence of priority_county
    def sort_key(feature):
        # Check if priority_county is in any position within the ar_ugcname list
        counties = feature['properties']['ar_ugcname']
        has_priority = any(priority_county in county for county in counties)
        # Return a tuple that first sorts by the presence of priority_county, then by the first county alphabetically
        return (not has_priority, counties[0])

    # Sort warnings with the sort function
    sorted_warnings = sorted(warnings['events']['features'], key=sort_key)

    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['WFO', 'Issuance Date', 'Issuance Time', 'Expire Time', 'Verified', 'Counties', 'Time Active (minutes)']) #header row in the csv

        for feature in sorted_warnings: #the data printed under each header
            wfo = feature['properties']['wfo']
            issue_time = parser.parse(feature['properties']['issue'])
            expire_time = parser.parse(feature['properties']['expire'])
            duration = calculate_duration(issue_time, expire_time)
            counties = ", ".join(feature['properties']['ar_ugcname'])
            verify = feature['properties'].get('verify', 'N/A')  # Fallback to 'N/A' if not present
            writer.writerow([wfo, issue_time.strftime('%Y-%m-%d'), issue_time.strftime('%I:%M:%S %p'), expire_time.strftime('%I:%M:%S %p'), verify, counties, duration])

# Main execution
warnings = get_tornado_warnings(wfo, start_date, end_date)
if warnings:
    csv_filename = "tornado_warningsUNTIMED.csv"
    save_to_csv(warnings, csv_filename, priority_county)
    print("Data saved to CSV.")
